In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler

In [ ]:
x = np.load('/content/drive/MyDrive/data.T/X.npy')
y = np.load('/content/drive/MyDrive/data.T/Y.npy')

In [ ]:
d = np.genfromtxt('/content/drive/MyDrive/data.T/Vert.csv', delimiter = ',')
d = d[1:]

In [ ]:
for i in range(15002):
    scaler = StandardScaler().fit(x[i])
    x[i] = scaler.transform(x[i])

In [ ]:
batch_size = 32
steps_per_epoch = len(x)//batch_size
steps_per_epoch

468

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import GRU
from keras.layers import SimpleRNN
from keras.metrics import BinaryAccuracy

In [ ]:
x = np.array([[[1,2,3],[4,5,6]],[[7,8,9],[10,11,12]]])
y = np.array([[[12,14,15],[16,20,18]],[[19,5,21],[22,23,24]]])
a = np.square(x-y)
print(a)
b = np.sum(a, axis = 2)
print('ааааа\n')
print(b)
print(np.mean(np.sqrt(b)))

[[[121 144 144]
  [144 225 144]]

 [[144   9 144]
  [144 144 144]]]
ааааа

[[409 513]
 [297 432]]
20.22288733810239


In [ ]:
import keras.backend as K

def my_loss(y_true, y_pred, d):
  y_coord = d[K.eval(K.argmax(y_true))] * 1000 
  y_pred_coord = d[K.eval(K.argmax(y_pred))] *1000
  loss = K.sqrt(K.sum(K.square(y_coord-y_pred_coord), axis = 2))
  return loss

def trick(d):
  def loss(y_true, y_pred):
    return my_loss(y_true, y_pred, d)
  return loss

In [ ]:
model = Sequential()
model.add(SimpleRNN(15002, return_sequences=True, activation="softmax"))
model.compile(optimizer='Adam',
loss=trick(d), run_eagerly=True)

In [ ]:
model.fit(x, y, epochs=10)

Epoch 1/10


InvalidArgumentError: ignored

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_1 (SimpleRNN)    (32, 4, 15002)            229665618 
                                                                 
Total params: 229,665,618
Trainable params: 229,665,618
Non-trainable params: 0
_________________________________________________________________


In [ ]:
test_x = np.load('/content/drive/MyDrive/data.T/testX.npy')
test_y = np.load('/content/drive/MyDrive/data.T/testY.npy')

In [ ]:
for i in range(250):
    scaler = StandardScaler().fit(test_x[i])
    test_x[i] = scaler.transform(test_x[i])

In [ ]:
y_pred= model.predict(test_x)

In [ ]:
k1=[]
k2=[]

In [ ]:
n_dip = 123

In [ ]:
for j in range(4):
  max = y_pred[n_dip][j][0]
  for i in range(15002):
    if y_pred[n_dip][j][i]>max:
      max = y_pred[n_dip][j][i]
      k = i
  k1.append(k)
  print(k)

13175
12442
12709
12169


In [ ]:
for j in range(4):
  max = test_y[n_dip][j][0]
  for i in range(15002):
    if test_y[n_dip][j][i]>max:
      max = test_y[n_dip][j][i]
      k = i
  k2.append(k)
  print(k)

13175
13057
12754
12718


In [ ]:
for i in range(4):
  tr = d[k2[i]]
  pr = d[k1[i]]
  err = (((tr[0]-pr[0])*1000)**2+((tr[1]-pr[1])*1000)**2+((tr[2]-pr[2])*1000)**2)**(1/2)
  print(err)

0.0
17.903365409888725
2.888309713309845
15.506715480719958


In [ ]:
for i in range(3):
  d1 = d[k1[i]]
  d2 = d[k1[i+1]]
  print(f'Между {i+1} и {i+2}')
  err = (((d2[0]-d1[0])*1000)**2+((d2[1]-d1[1])*1000)**2+((d2[2]-d1[2])*1000)**2)**(1/2)
  print(err)

Между 1 и 2
19.521415240704247
Между 2 и 3
19.36313079024154
Между 3 и 4
17.60251430904119


In [ ]:
err1 = []
err2 = []
for n_dip in range(len(test_y)):
  k1=[]
  k2=[]

  for j in range(4):
    max = y_pred[n_dip][j][0]
    for i in range(15002):
      if y_pred[n_dip][j][i]>max:
        max = y_pred[n_dip][j][i]
        k = i
    k1.append(k)

  for j in range(4):
    max = test_y[n_dip][j][0]
    for i in range(15002):
      if test_y[n_dip][j][i]>max:
        max = test_y[n_dip][j][i]
        k = i
    k2.append(k)
  
  for i in range(4):
    tr = d[k2[i]]
    pr = d[k1[i]]
    err1.append((((tr[0]-pr[0])*1000)**2+((tr[1]-pr[1])*1000)**2+((tr[2]-pr[2])*1000)**2)**(1/2))

  for i in range(3):
    d1 = d[k1[i]]
    d2 = d[k1[i+1]]
    err2.append((((d2[0]-d1[0])*1000)**2+((d2[1]-d1[1])*1000)**2+((d2[2]-d1[2])*1000)**2)**(1/2))

err1 = np.array( err1 )
err2 = np.array( err2 )

print(np.mean(err1))
print(np.mean(err2))

18.566256309933458
27.625611260248416


In [ ]:
model.save('SimpleRNN1_ep10_bs32.h5') 